In [1]:
from sklearn.svm import SVC
from sklearn.datasets import make_classification
import numpy as np
import os
import glob
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold

In [2]:
print(os.getcwd())
current = os.getcwd()
data_folder = os.path.join(current, "normalized_images")
chess_types_folders = glob.glob(os.path.join(data_folder, "*"))
# print(chess_types_folders)
pieces_info = []
labels = {"King": 1, "Knight":2, "Bishop":3, "Rook":4, "Pawn":5, "Queen":6}
for chess_types in chess_types_folders:
    pieces = glob.glob(f'{chess_types}/*')
    # print(pieces)
    type = chess_types.split("/")[-1]
    for piece in pieces:
        p = {"normalized_img": np.load(piece).reshape(-1), "label": labels[type]}
        pieces_info.append(p)
chess_df = pd.DataFrame(pieces_info)

/Users/giangto/Documents/umn/csci5525/5525Chess


In [3]:
print(chess_df.head())

                                      normalized_img  label
0  [0.2196078431372549, 0.2235294117647059, 0.227...      3
1  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...      3
2  [0.8666666666666667, 0.8666666666666667, 0.866...      3
3  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...      3
4  [0.6862745098039216, 0.6823529411764706, 0.690...      3


In [4]:
chess_df.iloc[0:5]['normalized_img'].shape #dim of 1 single image

(5,)

In [5]:
chess_df.iloc[0]['normalized_img']

array([0.21960784, 0.22352941, 0.22745098, ..., 0.76078431, 0.76078431,
       0.76078431], shape=(50176,))

In [6]:
#Load dataset
X, y = np.array(chess_df['normalized_img']), np.array(chess_df['label'])
X = np.array([x for x in X])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
#Selecting best C for Linear SVM Model
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		lin_svm = SVC(kernel='linear',C=C)
		lin_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = lin_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = lin_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)





C = 0.01: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 0.1: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 1.0: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 10: (avg train error, avg validation error) = (0.0000, 0.0000)
C = 100: (avg train error, avg validation error) = (0.0000, 0.0000)


In [19]:
#Fit linear model on best C
lowest_val_error_index = np.argmin(cv_val_avg_score)
C_chosen = C_values[lowest_val_error_index]

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='linear',C=C_chosen)
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"Linear SVM Performance Metrics with C={C_chosen}:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 0.01, with lowest avg test error: 0.0000
Final Test Error: 0.7938
Linear SVM Performance Metrics with C=0.01:
Accuracy: 0.2062
Precision: 0.2074
Recall: 0.2062
F1 Score: 0.2036

Confusion Matrix:
[[ 5  4  8  9  5  5]
 [ 8  6  7 15  7  2]
 [ 9  3 15  4  5  6]
 [ 3  8  8  7  4  8]
 [ 7 10 10  9 15  1]
 [ 6 12  8  4  9  5]]

Classification Report:
              precision    recall  f1-score   support

           1       0.13      0.14      0.14        36
           2       0.14      0.13      0.14        45
           3       0.27      0.36      0.31        42
           4       0.15      0.18      0.16        38
           5       0.33      0.29      0.31        52
           6       0.19      0.11      0.14        44

    accuracy                           0.21       257
   macro avg       0.20      0.20      0.20       257
weighted avg       0.21      0.21      0.20       257



In [22]:
#Selecting best C for RBF SVM Model (gamma=auto)
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		auto_svm = SVC(kernel='rbf',C=C, gamma='auto')
		auto_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = auto_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = auto_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)


C = 0.01: (avg train error, avg validation error) = (0.8097, 0.8097)
C = 0.1: (avg train error, avg validation error) = (0.8097, 0.8097)
C = 1.0: (avg train error, avg validation error) = (0.3205, 0.3206)
C = 10: (avg train error, avg validation error) = (0.0351, 0.0351)
C = 100: (avg train error, avg validation error) = (0.0000, 0.0000)


In [23]:
#Fit RBF model on best C (gamma='auto')
lowest_val_error_index = np.argmin(cv_val_avg_score)
C_chosen = C_values[lowest_val_error_index]

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='rbf',C=C_chosen, gamma='auto')
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C={C_chosen}, gamma='auto':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 100, with lowest avg test error: 0.0000
Final Test Error: 0.7237
RBF SVM Performance Metrics with C=100, gamma='auto':
Accuracy: 0.2763
Precision: 0.2732
Recall: 0.2763
F1 Score: 0.2727

Confusion Matrix:
[[ 4  3  9  3  6 11]
 [ 3 20  9  6  3  4]
 [10  4  9  9  4  6]
 [ 6  7  5 16  4  0]
 [ 5  8  8  7 18  6]
 [ 7  4 13  6 10  4]]

Classification Report:
              precision    recall  f1-score   support

           1       0.11      0.11      0.11        36
           2       0.43      0.44      0.44        45
           3       0.17      0.21      0.19        42
           4       0.34      0.42      0.38        38
           5       0.40      0.35      0.37        52
           6       0.13      0.09      0.11        44

    accuracy                           0.28       257
   macro avg       0.26      0.27      0.27       257
weighted avg       0.27      0.28      0.27       257



In [24]:
#Selecting best C for RBF SVM Model (gamma=scale)
C_values = [0.01, 0.1, 1.0, 10, 100]
cv_val_avg_score = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
	train_errors = []
	val_errors = []
	for train_idx, val_idx in skf.split(X_train, y_train):
		X_train_cv, X_val = X_train[train_idx], X_train[val_idx]
		y_train_cv, y_val = y_train[train_idx], y_train[val_idx]
		scale_svm = SVC(kernel='rbf',C=C, gamma='scale')
		scale_svm.fit(X_train, y_train)
		# validation error
		y_val_pred = scale_svm.predict(X_val)
		val_error = 1 - accuracy_score(y_val, y_val_pred)
		val_errors.append(val_error)

		# train error
		y_train_pred = scale_svm.predict(X_train_cv)
		train_error = 1 - accuracy_score(y_train_cv, y_train_pred)
		train_errors.append(train_error)

	avg_val_error = np.mean(val_errors)
	avg_train_error = np.mean(train_errors)

	# Report validation, test for each C
	print(f"C = {C}: (avg train error, avg validation error) = ({avg_train_error:.4f}, {avg_val_error:.4f})")

	cv_val_avg_score.append(avg_val_error)

C = 0.01: (avg train error, avg validation error) = (0.8097, 0.8097)
C = 0.1: (avg train error, avg validation error) = (0.8097, 0.8097)
C = 1.0: (avg train error, avg validation error) = (0.3205, 0.3206)
C = 10: (avg train error, avg validation error) = (0.0351, 0.0351)
C = 100: (avg train error, avg validation error) = (0.0000, 0.0000)


In [26]:
#Fit RBF model on best C (gamma='scale')
lowest_val_error_index = np.argmin(cv_val_avg_score)
C_chosen = C_values[lowest_val_error_index]

print(f"Best C = {C_chosen}, with lowest avg test error: {cv_val_avg_score[lowest_val_error_index]:.4f}")


final_model = SVC(kernel='rbf',C=C_chosen, gamma='scale')
final_model.fit(X_train, y_train)

y_test_pred = final_model.predict(X_test)
final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C={C_chosen}, gamma='scale':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Best C = 100, with lowest avg test error: 0.0000
Final Test Error: 0.7237
RBF SVM Performance Metrics with C=100, gamma='scale':
Accuracy: 0.2763
Precision: 0.2732
Recall: 0.2763
F1 Score: 0.2727

Confusion Matrix:
[[ 4  3  9  3  6 11]
 [ 3 20  9  6  3  4]
 [10  4  9  9  4  6]
 [ 6  7  5 16  4  0]
 [ 5  8  8  7 18  6]
 [ 7  4 13  6 10  4]]

Classification Report:
              precision    recall  f1-score   support

           1       0.11      0.11      0.11        36
           2       0.43      0.44      0.44        45
           3       0.17      0.21      0.19        42
           4       0.34      0.42      0.38        38
           5       0.40      0.35      0.37        52
           6       0.13      0.09      0.11        44

    accuracy                           0.28       257
   macro avg       0.26      0.27      0.27       257
weighted avg       0.27      0.28      0.27       257



RBF SVM with gamma = scale and gamma = auto performs comparably okay 
RBF SVM performs a lot more better with Linear SVM

In [8]:
#implement own version of SVM and tweak something?????
#DIY SVM
import numpy as np

class MultiClassSVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000, gamma=0.1, C=1.0):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.gamma = gamma
        self.C = C
        self.models = {}  # One binary SVM for each class

    def rbf_kernel(self, x, y):
        return np.exp(-self.gamma * np.linalg.norm(x - y) ** 2)

    def fit(self, X, y):
        self.classes = np.unique(y)
        for cls in self.classes:
            # Create binary labels for one-vs-rest
            y_binary = np.where(y == cls, 1, -1)
            svm = SVM(self.lr, self.lambda_param, self.n_iters, self.gamma, self.C)
            svm.fit(X, y_binary)
            self.models[cls] = svm

    def predict(self, X):
        scores = {}
        for cls, svm in self.models.items():
            scores[cls] = svm.predict_decision_scores(X)  # Decision scores from each SVM
        # Return the class with the highest decision score
        return np.array([max(scores, key=lambda k: scores[k][i]) for i in range(len(X))])

class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000, gamma=0.1, C=1.0):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.gamma = gamma
        self.C = C
        self.alpha = None
        self.b = 0
        self.support_vectors = None
        self.support_labels = None

    def rbf_kernel(self, x, y):
        return np.exp(-self.gamma * np.linalg.norm(x - y) ** 2)

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y = np.where(y <= 0, -1, 1)  # Convert labels to -1 and 1

        # Initialize alpha (Lagrange multipliers)
        self.alpha = np.zeros(n_samples)
        self.b = 0

        # Gradient descent for alpha optimization
        for _ in range(self.n_iters):
            for i in range(n_samples):
                # Calculate decision function using the RBF kernel
                decision = 0
                for j in range(n_samples):
                    decision += self.alpha[j] * y[j] * self.rbf_kernel(X[i], X[j])
                decision += self.b

                # Update alpha and b if constraint is violated
                if y[i] * decision < 1:
                    self.alpha[i] += self.lr * (1 - y[i] * decision)

                    # Clip alpha to stay within [0, C]
                    self.alpha[i] = np.clip(self.alpha[i], 0, self.C)

                    self.b += self.lr * y[i]

        # Store support vectors (non-zero alpha values)
        support_vector_indices = self.alpha > 1e-5
        self.support_vectors = X[support_vector_indices]
        self.support_labels = y[support_vector_indices]
        self.alpha = self.alpha[support_vector_indices]

    def predict(self, X):
        predictions = []
        for x in X:
            decision = 0
            for alpha, sv, sv_label in zip(self.alpha, self.support_vectors, self.support_labels):
                decision += alpha * sv_label * self.rbf_kernel(x, sv)
            decision += self.b
            predictions.append(np.sign(decision))
        return np.array(predictions)

    def predict_decision_scores(self, X):
        scores = []
        for x in X:
            decision = 0
            for alpha, sv, sv_label in zip(self.alpha, self.support_vectors, self.support_labels):
                decision += alpha * sv_label * self.rbf_kernel(x, sv)
            decision += self.b
            scores.append(decision)
        return np.array(scores)


In [9]:
diy_svm = MultiClassSVM()
diy_svm.fit(X_train, y_train)
y_test_pred = diy_svm.predict(y_test)

final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Final Test Error: {final_test_error:.4f}")
print(f"RBF SVM Performance Metrics with C={C_chosen}, gamma='scale':")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

KeyboardInterrupt: 